In [ ]:

####################################################################################################
####################################################################################################

import datetime
import sys
import os.path

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd

from IPython.display import display
from PIL import Image

import tensorflow as tf

print (tf.__version__)

import keras
import keras.utils.vis_utils
from keras import backend as K
from keras import layers
from keras import regularizers
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization
from keras.models import load_model
from keras.models import Sequential
from keras.utils import plot_model

sys.path.append(os.path.abspath("../"))
from xy_s2integrals_models import *
from helpers import *

sys.path.append(os.path.abspath("../../"))
from pax_utils import s1s2_utils

pd.set_option('display.max_columns', 500)


In [ ]:

####################################################################################################
####################################################################################################

nEventsTrain = 100000
#nEventsTrain = 1000


####################################################################################################
####################################################################################################

#file_all     = "../../data_lfs/merged_all_200k_1803261941.pkl"
#file_all     = "../../data_lfs/merged_all_200k_1803261941.pkl"
#file_all     = "../../pax_merge/merged/apr26/merged_all_200000.pkl"
file_all     = "../../pax_merge/merged/apr30/merged_all_200000.pkl"


df_all   = pd.read_pickle(file_all)
df_all   = df_all[df_all['intr_count'] == 1].reset_index(drop=True)
df_all   = df_all[0:nEventsTrain][:]


####################################################################################################
####################################################################################################

s2_cols = s1s2_utils.getS2integralsDataFrameColumns()

df_train_input = df_all[:][s2_cols]
df_train_truth = df_all[:][['x', 'y']]


        
####################################################################################################
####################################################################################################

#checkTrainingData(df_train_input, df_train_truth)



In [ ]:

##########################################################################################
##########################################################################################

lst_models = [
    #Xenon1TPosRecMLP(),
    #bargeModel()
    xy_s2integrals_dnnModel()
]



In [ ]:

##########################################################################################
##########################################################################################

for iModel in range(0, len(lst_models)):
    
    ######################################################################################
    ######################################################################################

    model, model_name = lst_models[iModel]
  
    folder   = "models/"    
    name     = 'model' + '_' + model_name + '_may02'
    name_h5  = folder + name + ".h5"
    name_png = folder + name + ".png"

    
    ######################################################################################
    ######################################################################################

    plot_model(model, to_file=name_png, show_layer_names=True, show_shapes=True)

    trainable_count     = int(np.sum([K.count_params(p) for p in set(model.trainable_weights)]))
    non_trainable_count = int(np.sum([K.count_params(p) for p in set(model.non_trainable_weights)]))

    print()
    display(Image.open(name_png))
    print()
    assert(non_trainable_count == 0)
    #print("Model Summary:")
    #summary = model.summary()
    #print()


    ######################################################################################
    # Fit Model
    ######################################################################################
    
    epochs     = 10
    batch_size = 64
    verbose    = 1
    
    model.fit(df_train_input.values, df_train_truth.values, epochs=epochs, batch_size=batch_size, verbose=verbose)

             
    ######################################################################################
    # Save Model
    ######################################################################################
    
    model.save(name_h5, overwrite=True)
    
    
    ######################################################################################
    ######################################################################################
    
    continue
    
    